### 02 - Create Lakehouse Table (Delta Parquet) from CSV files 
This is second Notebook of the solution and uses CSV files downloaded in the previous step to create Lakehouse Table (Silver Layer) to be  used in the subsequent step for building the Gold Layer (Star Schema) tables for reporting.

In [1]:
%run Utils

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 4, Finished, Available, Finished)

In [2]:
config_dict = get_config_dict()

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 5, Finished, Available, Finished)

In [3]:
source_dir = "Files/cms_raw"
sourct_dir_full_abfss_path = get_full_abfss_path(config_dict['workspace_id'], config_dict['lakehouse_id'], source_dir) 

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 6, Finished, Available, Finished)

In [4]:
import os
from urllib.parse import urlparse

# function to extract the name of the file form the file full path
# name of the file represents year value for the data for that file 
# example file_path "abfss://f16554c0-3959-4537-9128-ad08c6ac1692@onelake.dfs.fabric.microsoft.com/96391b77-cc32-4714-b791-70a15ccc429f/Files/cms_raw/2013.csv?version=1727404441686?flength=3375263337"
# year value is 2013 which is name of the file
def extract_year_from_file_name(path):
    # Parse the URL to get the path
    parsed_url = urlparse(path)
    # Extract the file name with extension
    file_name_with_extension = os.path.basename(parsed_url.path)
    # Remove the extension
    file_name, extension = os.path.splitext(file_name_with_extension)
    return int(file_name)

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 7, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

#create UDF wrapper around the function which extracts name of the file
extract_year_from_file_name_udf = udf(extract_year_from_file_name, IntegerType())

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 8, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import input_file_name, regexp_extract

# load pyspark dataframe with data from all csv files "Files/cms_raw/2013.csv"
# add a column to the dataframe for the file_path using input_file_name() function 
df = spark.read.format("csv").option("header","true").load(sourct_dir_full_abfss_path + "/*.csv").withColumn("file_path", input_file_name())

# use the UDF to extract year value from the file path and add it as a column to data frame
df = df.withColumn("Year", extract_year_from_file_name_udf(df["file_path"]))
display(df)

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fb2e421c-89a3-4977-aff6-794137a922c0)

In [7]:
from pyspark.sql.types import LongType, DecimalType
from pyspark.sql.functions import lit, col, concat

#make minor transformation to set the appropriate data types for the various columns
#as well as add a few new columns based on existing columns
df = df.withColumn("Tot_Drug_Cst", df.Tot_Drug_Cst.cast(DecimalType(10,2))) \
        .withColumn("Tot_30day_Fills", df.Tot_30day_Fills.cast(DecimalType(10,2))) \
        .withColumn("GE65_Tot_30day_Fills", df.GE65_Tot_30day_Fills.cast(DecimalType(10,2))) \
        .withColumn("GE65_Tot_Drug_Cst", df.GE65_Tot_Drug_Cst.cast(DecimalType(10,2))) \
        .withColumn("Prscrbr_City_State", concat(df.Prscrbr_City, lit(", "), df.Prscrbr_State_Abrvtn)) \
        .withColumn("Prscrbr_Full_Name", concat(df.Prscrbr_Last_Org_Name, lit(", "), df.Prscrbr_First_Name)) \
        .withColumn("Tot_Clms", df.Tot_Clms.cast(LongType())) \
        .withColumn("Tot_Day_Suply", df.Tot_Day_Suply.cast(LongType())) \
        .withColumn("Tot_Benes", df.Tot_Benes.cast(LongType())) \
        .withColumn("GE65_Tot_Clms", df.GE65_Tot_Clms.cast(LongType())) \
        .withColumn("GE65_Tot_Benes", df.GE65_Tot_Benes.cast(LongType())) \
        .withColumn("GE65_Tot_Day_Suply", df.GE65_Tot_Day_Suply.cast(LongType())) \
        .drop("file_path")

display(df)

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f34a9c5-78ed-4660-b891-b0c591f3e9f5)

In [17]:
#write out the table to Lakehouse 
table_name = "cms_provider_drug_costs"
dest_dir = f"Tables/{table_name}"
dest_dir_full_abfss_path = get_full_abfss_path(config_dict['workspace_id'], config_dict['lakehouse_id'], dest_dir) 
print(dest_dir_full_abfss_path)
#df.write.mode("overwrite").format('delta').option('path', dest_dir_full_abfss_path).saveAsTable(table_name)

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 20, Finished, Available, Finished)

abfss://c8f59358-83d0-4711-8b57-ebc3c414b0a1@onelake.dfs.fabric.microsoft.com/93c691f7-1409-44ef-ab45-fa3a3fb74289/Tables/cms_provider_drug_costs


In [22]:
df.write.mode("overwrite").format('delta').save(dest_dir_full_abfss_path)
#df.write.mode("overwrite").format('delta').saveAsTable("cms_lakehouse2.cms_provider_drug_costs")

StatementMeta(, 21b5e594-1b2b-4e97-b596-683d1522bde1, 25, Finished, Available, Finished)